In [1]:
# handy imports to have
%matplotlib inline
from sympy import symbols, Eq, solve
import math
from itertools import combinations_with_replacement , permutations, combinations, count, chain
import numpy as np
import time
import functools
from tqdm.notebook import trange, tqdm
import ipywidgets as widgets
import matplotlib.pyplot as plt
from collections import Counter, deque
import more_itertools as mit
import string
import ast

In [78]:
# sample input

input_map = [[[2, 18], [-2, 15]], [[9, 16], [10, 16]], [[13, 2], [15, 3]], [[12, 14], [10, 16]], [[10, 20], [10, 16]], [[14, 17], [10, 16]], [[8, 7], [2, 10]], [[2, 0], [2, 10]], [[0, 11], [2, 10]], [[20, 14], [25, 17]], [[17, 20], [21, 22]], [[16, 7], [15, 3]], [[14, 3], [15, 3]], [[20, 1], [15, 3]] ]

In [85]:
# puzzle input, part 1 = 4886370

input_map = [[[220580, 684270], [436611, 263737]], [[3329538, 3016377], [3355914, 2862466]], [[2605308, 2023938], [2197530, 2271330]], [[1810202, 3423309], [1829362, 3182862]], [[480296, 3999646], [1694700, 4178942]], [[46556, 1283362], [ -91140, 1441882]], [[3741660, 3959257], [3537901, 3368697]], [[3399994, 700264], [3748004, 2000000]], [[1531981, 3801761], [1694700, 4178942]], [[193367, 2712458], [ -91140, 1441882]], [[3199067, 2194575], [3748004, 2000000]], [[1878117, 2578817], [2197530, 2271330]], [[2439089, 3168242], [1829362, 3182862]], [[273443, 171076], [436611, 263737]], [[3680413, 2477027], [3748004, 2000000]], [[3620241, 2904998], [3355914, 2862466]], [[1728351, 2895399], [1829362, 3182862]], [[1894207, 1168355], [2197530, 2271330]], [[856867, 3271314], [1829362, 3182862]], [[3056788, 2626224], [3355914, 2862466]], [[3598024, 3322247], [3537901, 3368697]], [[1662543, 3128823], [1829362, 3182862]], [[3992558, 1933059], [3748004, 2000000]], [[1844282, 2994285], [1829362, 3182862]], [[3604375, 3668021], [3537901, 3368697]], [[2569893, 3911832], [1694700, 4178942]], [[117970, 37503], [436611, 263737]], [[3951385, 3125577], [3537901, 3368697]], [[2482373, 2648092], [2197530, 2271330]], [[915040, 1835970], [ -91140, 1441882]], [[3047883, 3301452], [3537901, 3368697]], [[117432, 1503889], [ -91140, 1441882]], [[1136011, 261705], [436611, 263737]], [[2343111, 66183], [2081841,  -807749]], [[608229, 955721], [436611, 263737]], [[1189379, 3999750], [1694700, 4178942]], [[766640, 26597], [436611, 263737]], [[3891093, 2110588], [3748004, 2000000]]]

In [46]:
# process input, get max sizes (this is only to confirm understanding, apparently max is HUGE and eats too much memory.
# will re-write below without using a matrix (likely a set so duplicates get handled nicely)
# was nice to confirm my assumptions though

x_max = 0
y_max = 0
x_min = 0
y_min = 0

for pair in input_map:
    [sx, sy],[bx, by] = pair
    x_max = max(x_max, sx, bx)
    y_max = max(y_max, sy, by)
    x_min = min(x_min, sx, bx)
    y_min = min(y_min, sy, by)
    
if x_min <= 0:
    x_min = -1 * x_min
    x_min += 20
if y_min <= 0:
    y_min = -1 * y_min
    y_min += 20
    
x_max += 1 + x_min + 20
y_max += 1 + y_min + 20

print('x max:', x_max, '| x min:', x_min, '|| y max:', y_max, '| y min:', y_min)

# build initial map

signal_map = ['' for i in range(0, y_max)]
for it, line in enumerate(signal_map):
    signal_map[it] = ['.' for i in range(0, x_max)]
    
print_map()

print('-------------------')


# populate map with sensors and beacons
# populate sensors and beacons

for pair in input_map:
    [sx, sy],[bx, by] = pair
    sx += x_min
    sy += y_min
    bx += x_min
    by += y_min
    signal_map[sy][sx] = 'S'
    signal_map[by][bx] = 'B'
    plot_coverage(sx, sy, bx, by)
    
print_map()

examine = signal_map[10+y_min]

count = 0
for c in examine:
    if c != '.' and c != 'B' and c != 'S':
        count += 1
        
print(count)


x max: 68 | x min: 22 || y max: 63 | y min: 20
000 ....................................................................
001 ....................................................................
002 ....................................................................
003 ....................................................................
004 ....................................................................
005 ....................................................................
006 ....................................................................
007 ....................................................................
008 ....................................................................
009 ....................................................................
010 ....................................................................
011 ....................................................................
012 ....................................................................
013 

In [44]:
def print_map(doit=False):
    if len(signal_map) > 100 and not doit:
        print('Map too large to display')
        return
    for i, line in enumerate(signal_map):
        string = ''
        for s in line:
            string += s
        print('%03d' % (i), string)

In [ ]:
def plot_coverage(sx, sy, bx, by):
    
    distance = abs(sy - by) + abs(sx - bx)
    
    for yc in range(sy-distance, sy+distance+1):
        y_offset = distance - abs(yc - sy)
        for xc in range(sx - y_offset, sx+y_offset+1):
            if 0 <= xc < len(signal_map[0]) and 0 <= yc < len(signal_map):
                if signal_map[yc][xc] == '.':
                    signal_map[yc][xc] = '#'
            
            
    




plot_coverage(10, 7, 4, 10)
print_map()

In [20]:
# problem one solution:

x_max = 0
y_max = 0
x_min = 0
y_min = 0

for pair in input_map:
    [sx, sy],[bx, by] = pair
    x_max = max(x_max, sx, bx)
    y_max = max(y_max, sy, by)
    x_min = min(x_min, sx, bx)
    y_min = min(y_min, sy, by)
    
if x_min <= 0:
    x_min = -1 * x_min
    x_min += 20
if y_min <= 0:
    y_min = -1 * y_min
    y_min += 20
    
x_max += 1 + x_min + 20
y_max += 1 + y_min + 20

print('x max:', x_max, '| x min:', x_min, '|| y max:', y_max, '| y min:', y_min)


# evaluate beacons and sensors, only mapping on the line we care about

l2s = y_min + 2000000   # line to study, 10 is sample and 2000000 is problem
x_used = set()     # set to store used x coords (y fixed in line above)
x_non_used = set()    # count of B and S on the line to ignore

print('Line To Study:', l2s)

for pair in input_map:
    [sx, sy],[bx, by] = pair
    sx += x_min
    sy += y_min
    bx += x_min
    by += y_min
    if sy == l2s:
        x_non_used.add(sx)
    if by == l2s:
        x_non_used.add(bx)
    calculate_coverage(sx, sy, bx, by, l2s)
    
print('x_used:', len(x_used))
print('x_not_used:', len(x_non_used))

answer_set = x_used - x_non_used

print('answer:', len(answer_set))
    



x max: 4083739 | x min: 91160 || y max: 4986732 | y min: 807769
Line To Study: 2807769
x_used: 4886371
x_not_used: 1
answer: 4886370


In [17]:
def calculate_coverage(sx, sy, bx, by, l2s):
    
    distance = abs(sy - by) + abs(sx - bx)
    
    for yc in range(sy-distance, sy+distance+1):
        if yc == l2s:
            y_offset = distance - abs(yc - sy)
            for xc in range(sx - y_offset, sx+y_offset+1):
                x_used.add(xc)

In [ ]:
# part 2 attempts - this is a failure. too much memory. Even tried slicing it into chunks, still going to take hours. New approach below.
signal_map = ['' for i in range(0, 10000+1)]
for i in range(0, 10000+1):
    signal_map[i] = ['.' for i in range(0, 10000+1)]

for iterator1 in range(0, 4000000, 10000):
    for iterator2 in range(0, 4000000, 10000):
        y_low_limit = iterator1
        y_upper_limit = iterator1+10000
        y_range = y_upper_limit - y_low_limit

 sensors_and_beacons   x_low_limit = iterator2
        x_upper_limit = iterator2 + 10000
        x_range = x_upper_limit - x_low_limit

        print('------------------- y:', iterator1, 'x:', iterator2 )


        # populate map with sensors and beacons

        for pair in tqdm(input_map, leave=False):
            [sx, sy],[bx, by] = pair
            if x_low_limit <= sx <= x_upper_limit and y_low_limit <= sy <= y_upper_limit:
                signal_map[sy-y_low_limit][sx-x_low_limit] = 'S'
            if x_low_limit <= bx <= x_upper_limit and x_low_limit <= by <= x_upper_limit:
                signal_map[by-y_low_limit][bx-x_low_limit] = 'B'
            plot_coverage_part_2(sx, sy, bx, by, x_low_limit, x_upper_limit, y_low_limit, y_upper_limit)




        for i in range(0, y_range):
            for j in range(0, x_range):
                if signal_map[j][i] == '.':
                    print('Winner at :', j, ',', i, 'for score of ', (j*4000000)+i)
                    break
                else:
                    signal_map[j][i] = '.'

In [3]:
def plot_coverage_part_2(sx, sy, bx, by, x_low, x_high, y_low, y_high):
    
    distance = abs(sy - by) + abs(sx - bx)
    
    y_start = max(sy-distance, y_low)
    y_end = min(sy+distance+1, y_high+1)
    
    for yc in range(y_start, y_end):
            y_offset = distance - abs(yc - sy)
            x_start = max(sx - y_offset, x_low)
            x_end = min(sx+y_offset+1, x_high+1)
            for xc in range(x_start, x_end):
                    if signal_map[yc-y_low][xc-x_low] == '.':
                        signal_map[yc-y_low][xc-x_low] = '#'


In [14]:
#Part 2, take 3 (i think?) This time we go every coord and see if it's in range of a sensor

# new approach also going to take forever. This is not fun. Lets try again below

sensors_and_beacons = set()
sense_range = []


for pair in input_map:
    [sx, sy],[bx, by] = pair
    sensors_and_beacons.add((sx, sy))
    sensors_and_beacons.add((bx, by))
    distance = abs(sy - by) + abs(sx - bx)
    sense_range.append(((sx, sy), distance))
    

for i in tqdm(range(14, 4000000), initial=14):
    for j in tqdm(range(0, 4000000), leave=False):
        
        covered = False
        for ((sx, sy), s_range) in sense_range:
            distance = abs(sy-i) + abs(sx-j)
            if s_range > distance:
                covered = True
                break
        if not covered:
            if (j, i) not in sensors_and_beacons:
                print('Winner is ', j, ',', i, ' -> ', (j*4000000)+i )
    
            


  0%|          | 14/3999986 [00:00<?, ?it/s]

  0%|          | 0/4000000 [00:00<?, ?it/s]

  0%|          | 0/4000000 [00:00<?, ?it/s]

  0%|          | 0/4000000 [00:00<?, ?it/s]

  0%|          | 0/4000000 [00:00<?, ?it/s]

...

  0%|          | 0/4000000 [00:00<?, ?it/s]

  0%|          | 0/4000000 [00:00<?, ?it/s]

  0%|          | 0/4000000 [00:00<?, ?it/s]

  0%|          | 0/4000000 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
# part 2, attempt 4 or 5. 
# Lets try allocating (still a LOT of) memory, but only one axis worth. 
# N rather than N^2. Then do range overlap comparisons over that 4000000 unit list.

# I'm still leery about the memory usage but we'll see how this one goes.

# YAY! Took about 15 seconds to solve. Ugh this one was both fun and super annoying at the same time.

In [86]:
t_range = 4000001

signal_map = [[] for _ in range(0, t_range+1)]

for pair in tqdm(input_map):
    [sx, sy],[bx, by] = pair
    append_ranges(sx, sy, bx, by)

  0%|          | 0/38 [00:00<?, ?it/s]

In [54]:
def append_ranges(sx, sy, bx, by):
    
    distance = abs(sy - by) + abs(sx - bx)
    
    y_start = max(sy-distance, 0)
    y_end = min(sy+distance+1, t_range+1)
    
    for yc in range(y_start, y_end):
            y_offset = distance - abs(yc - sy)
            x_start = max(sx - y_offset, 0)
            x_end = min(sx+y_offset, t_range)
            signal_map[yc].append((x_start, x_end))
            


In [87]:
def calculate_coverage(yc):
    coverages = signal_map[yc]
    coverages.sort()
    
    start = coverages[0][0]
    end = coverages[0][1]
    
    for s, e in coverages:
        if start <= s <= end+1:
            if end < e:
                end = e

    
    if start != 0 or end < t_range:
        print('Trouble on line ', yc)
        print(yc, start, end)           
    
calculate_coverage(2948438)

Trouble on line  2948438
2948438 0 2843632


In [89]:
#signal_map[2948438]
print( 2843633*4000000 + 2948438) 

#problem on line 2948438, the missing coord is 2843633 - correct answer is 11374534948438! YAY!

11374534948438


In [88]:
for yc in tqdm(range(0, len(signal_map))):
    calculate_coverage(yc)

  0%|          | 0/4000002 [00:00<?, ?it/s]

Trouble on line  2948438
2948438 0 2843632
